# 03_feature_engineering.ipynb

## 1. 目的
EDA(01〜02)で得た知見をもとに、モデル入力用の特徴量を作成する。


## 2. データ読み込み


In [ ]:
import pandas as pd

In [4]:
import pandas as pd

df = pd.read_csv("data/processed/eda_output.csv")
df.head()


,EmployeeID,recorddate_key,birthdate_key,orighiredate_key,terminationdate_key,age,length_of_service,city_name,department_name,job_title,store_name,gender_short,gender_full,termreason_desc,termtype_desc,STATUS_YEAR,STATUS,BUSINESS_UNIT,age_group,attrition_flag
0,1318,12/31/2006 0:00,1/3/1954,8/28/1989,1/1/1900,52,17,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2006,ACTIVE,HEADOFFICE,50代,0
1,1318,12/31/2007 0:00,1/3/1954,8/28/1989,1/1/1900,53,18,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2007,ACTIVE,HEADOFFICE,50代,0
2,1318,12/31/2008 0:00,1/3/1954,8/28/1989,1/1/1900,54,19,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2008,ACTIVE,HEADOFFICE,50代,0
3,1318,12/31/2009 0:00,1/3/1954,8/28/1989,1/1/1900,55,20,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2009,ACTIVE,HEADOFFICE,50代,0
4,1318,12/31/2010 0:00,1/3/1954,8/28/1989,1/1/1900,56,21,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2010,ACTIVE,HEADOFFICE,50代,0


## 3. 特徴量作成


### 若年女性フラグ（is_young_female）
（Step3 性別ごとの離職傾向）  
男女で離職率に差があり、女性の方が約1.1ポイント高い（約3.5％ vs 2.4％）。  
離職者のうち女性が約6割を占めるという結果からも、若年女性層の離職リスクが比較的高いと考えられる。  
  
これは、育児・時短勤務・キャリア形成支援などの環境要因が影響している可能性がある。  
→ 年代と性別の掛け合わせで「若年女性層」の離職傾向を特徴量化。

In [7]:
# 若年女性フラグ（20〜30代女性は離職リスクが高い傾向）
df["is_young_female"] = (
    (df["gender_short"] == "F") &
    (df["age_group"].isin(["20代以下", "30代"]))
).astype(int)

df[["is_young_female"]]

,is_young_female
0,0
1,0
2,0
3,0
4,0
...,...
49648,0
49649,1
49650,1
49651,1


### 高齢男性フラグ（is_senior_male）
（Step5・Step6 年齢別×性別の分析）

60代以上の離職率は20％超で、定年退職が主な要因。  
特に男性は28％と高く、定年制度のような構造による離職要因を反映している。    
これをフラグ化することで、定年層の自然離職と中途離職を区別でき、
モデルが「予防できる離職」と「不可避の離職」を学習上で切り分けられる。

In [10]:
# 高齢男性フラグ（定年による離職リスクを表す）
df["is_senior_male"] = (
    (df["gender_short"] == "M") &
    (df["age_group"] == "60代以上")
).astype(int)

### 部署内の相対勤続年数（relative_service）
（Step5 年齢と勤続年数の関係）  
EDAで「勤続年数が安定している層（30〜50代）」が明確に存在。  
部署ごとに平均勤続年数が異なるため、
その人が部署の中でどれくらいベテランかを見ることができる。

同じ10年勤続でも「平均5年の部署」と「平均15年の部署」では意味が異なるため、
「部署内での相対的な位置（＝定着度）」を指標化。

In [11]:
# 部署内の相対勤続年数（部署平均との比率）
dept_mean_service = df.groupby("department_name")["length_of_service"].transform("mean")
df["relative_service"] = df["length_of_service"] / (dept_mean_service + 1e-6)


### 年齢カテゴリ（age_group_encoded）
（Step5 年代と離職率の傾向）

離職率には年代ごとに明確な構造差があり、  
20代以下：やや離職率高い（早期離職）  
30〜50代：安定して定着  
60代以上：定年要因で高い  

このように年齢層が順序的に並んでおり、年代ごとに影響があるため特徴を作成

In [12]:
# 年齢カテゴリを順序符号化（構造的傾向を反映）
df["age_group_encoded"] = df["age_group"].map({
    "20代以下": 0,
    "30代": 1,
    "40代": 2,
    "50代": 3,
    "60代以上": 4
})


In [13]:
df.to_csv("data/processed/employee_with_features.csv", index=False)